# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [5]:
from pymongo import MongoClient
client = MongoClient("mongo")
database = client['exercices']
collection = database['youtube']

In [6]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [7]:
df_youtube['date'] = pd.to_datetime(df_youtube['date'], errors='coerce')
df_youtube = df_youtube.dropna(subset=['date'])


df_youtube.rename(columns={'video_id':'_id'}, inplace=True)

df_youtube = df_youtube.drop_duplicates(subset='_id', keep='last')

l_d =df_youtube.to_dict(orient='records')

import json
with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)

for video in l_d:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']

### Importer les données

In [8]:
collection.insert_many(l_d)

## Question 1  

In [11]:
cursor = collection.find({"channel_title":"Apple"})
list(cursor)

[{'_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 19707391,
  'likes': 381919,
  'dislikes': 69465,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Science & Technology'},
 {'_id': 'A9k88sMyiJM',
  'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|airpods|apple watch|apple watch series 3|apple watch cellular|apple watch 4g|apple music|40 million songs on your wrist|apple skateboard ad|apple train station ad|apple roll ad|apple watch skateboard ad|apple watch train station ad|apple watch roll ad|airpods skateboard ad|airpods train station ad|airpods roll ad|apple music skat

## Question 2

In [23]:
cursor = collection.distinct("category_id")
print(len(cursor))

16


## Question 3

In [37]:
for elt in l_d:
    collection.update_one({"_id":elt["_id"]}, {"$set":{"tags_new": elt['tags'].split("|")}}, upsert=True)

In [38]:
collection.find_one()

{'_id': '5ywKal6-anc',
 'title': 'Gigi Hadid Loses High Heel During Fashion Week, Walks It Off Like a Pro | TMZ',
 'channel_title': 'TMZ',
 'category_id': 24,
 'tags': 'TMZ2016FS11221|TMZ|Hollywood|Celebrity|Entertainment|Famous|Hollywood News|Fame|Entertainment News',
 'views': 703750,
 'likes': 2921,
 'dislikes': 2196,
 'comment_total': 1042,
 'thumbnail_link': 'https://i.ytimg.com/vi/5ywKal6-anc/default.jpg',
 'date': datetime.datetime(1970, 1, 1, 0, 0),
 'category_name': 'Entertainment',
 'tags_new': ['TMZ2016FS11221',
  'TMZ',
  'Hollywood',
  'Celebrity',
  'Entertainment',
  'Famous',
  'Hollywood News',
  'Fame',
  'Entertainment News']}

## Question 4

In [24]:
cursor = collection.find().sort("views", -1).limit(5)
list(cursor)

[{'_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Music'},
 {'_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags': 'Dusk Till Dawn|Pop|RCA Records Label|ZAYN feat. Sia',
  'views': 36323498,
  'likes': 1431683,
  'dislikes': 28049,
  'comment_total': 100661,
  'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg',
  'date': datetime.datetime(1970, 1, 1, 0, 0),
  'category_name': 'Music'},
 {'_id': 'LunHybOKIjU',
  'title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher',
  'channel_title': 'BETNetworks',
  'category_id': 24,
  'tags': 'BET|BET Netwo

## Question 5

In [25]:
cursor = collection.aggregate([{"$group":{"_id":"$category_name","nombre de vues en moyenne":{"$avg":"$views"}}}])
list(cursor)

[{'_id': 'Film & Animation', 'nombre de vues en moyenne': 966547.2871287129},
 {'_id': 'Autos & Vehicles', 'nombre de vues en moyenne': 622727.8648648649},
 {'_id': 'Nonprofits & Activism', 'nombre de vues en moyenne': 681178.2},
 {'_id': 'Gaming', 'nombre de vues en moyenne': 777213.0357142857},
 {'_id': 'Shows', 'nombre de vues en moyenne': 8726.0},
 {'_id': 'Pets & Animals', 'nombre de vues en moyenne': 633906.4210526316},
 {'_id': 'Music', 'nombre de vues en moyenne': 1442240.943283582},
 {'_id': 'Sports', 'nombre de vues en moyenne': 618074.711409396},
 {'_id': 'Education', 'nombre de vues en moyenne': 550585.2659574468},
 {'_id': 'Entertainment', 'nombre de vues en moyenne': 1059427.7735849055},
 {'_id': 'Travel & Events', 'nombre de vues en moyenne': 426492.22222222225},
 {'_id': 'Science & Technology',
  'nombre de vues en moyenne': 861563.6158940397},
 {'_id': 'People & Blogs', 'nombre de vues en moyenne': 892992.28515625},
 {'_id': 'News & Politics', 'nombre de vues en moyenn

## Question 6 

In [30]:
cursor = collection.aggregate([{"$group":{"_id":"$channel_title", "nombre de like moyen":{"$avg":"$likes"}}}])
list(cursor)

[{'_id': 'The Slow Mo Guys', 'nombre de like moyen': 50848.0},
 {'_id': 'Focus Features', 'nombre de like moyen': 194.0},
 {'_id': 'Ed Gillespie', 'nombre de like moyen': 0.0},
 {'_id': 'SumerianRecords', 'nombre de like moyen': 86275.0},
 {'_id': 'Manto kay SO Afsanay', 'nombre de like moyen': 25.0},
 {'_id': 'FreeDawkins', 'nombre de like moyen': 989.6666666666666},
 {'_id': 'FilmRise Releasing', 'nombre de like moyen': 1213.0},
 {'_id': 'LiamPayneVEVO', 'nombre de like moyen': 181526.0},
 {'_id': 'Томикс-ТВ online', 'nombre de like moyen': 2268.0},
 {'_id': 'Sacramento Kings', 'nombre de like moyen': 1016.0},
 {'_id': 'Inside Edition', 'nombre de like moyen': 3249.6},
 {'_id': 'Kat Napiorkowska', 'nombre de like moyen': 3335.0},
 {'_id': 'Elevation Worship', 'nombre de like moyen': 1552.0},
 {'_id': 'Kerry Wong', 'nombre de like moyen': 83.0},
 {'_id': 'Micah Elizabeth Scott', 'nombre de like moyen': 261.0},
 {'_id': 'Amazing Timelapse', 'nombre de like moyen': 12.0},
 {'_id': 'U.S.